
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/77_Prompt_Engineering_CoT_ToT.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🌳 Chain of Thought (CoT) & Tree of Thoughts (ToT)

Modele językowe działają na zasadzie "następnego słowa". Nie mają planu.
Jeśli zapytasz: *"Rozwiąż to równanie"*, model zacznie pisać wynik, zanim w ogóle go obliczy.

**Ewolucja Promptingu:**
1.  **Standard:** `Q: Ile to 2+2? A: 4` (Zero myślenia).
2.  **Chain of Thought (CoT):** `Q: Ile to 2+2? A: Myślmy krok po kroku. Mam dwie liczby...` (Model zyskuje czas obliczeniowy).
3.  **Tree of Thoughts (ToT):** Algorytm przeszukiwania (BFS/DFS) na myślach modelu.

**Algorytm ToT:**
1.  **Dekompozycja:** Rozbij problem na kroki.
2.  **Generator:** Wymyśl 3 możliwe kolejne kroki.
3.  **Ewaluator:** Oceń każdy krok (np. "To wygląda obiecująco" vs "To ślepa uliczka").
4.  **Search:** Idź dalej tylko najlepszą ścieżką.

Zastosujemy to do **Gry w 24** (Masz liczby 4, 9, 10, 13. Użyj +, -, *, /, żeby otrzymać 24).

In [1]:
import operator

# 1. DEFINIUJEMY PROBLEM (Gra w 24)
# Mamy liczby. Musimy znaleźć działanie, które da 24.
# Przykład dla ludzi: 4, 9, 10, 13
# Rozwiązanie: (13 - 9) * (10 - 4) = 4 * 6 = 24

input_numbers = [4, 9, 10, 13]
TARGET = 24

print(f"Liczby wejściowe: {input_numbers}")
print(f"Cel: {TARGET}")

Liczby wejściowe: [4, 9, 10, 13]
Cel: 24


## Symulacja LLM (Generator Myśli)

Normalnie użylibyśmy tu GPT-4, żeby wymyślał działania.
Tu napiszemy funkcję w Pythonie, która **symuluje** działanie modelu – generuje wszystkie możliwe operacje matematyczne między dwiema liczbami.

To będzie nasz **Generator Myśli**.

In [2]:
def get_possible_next_steps(current_numbers):
    """
    To jest symulacja tego, co robiłby GPT-4 w kroku 'Generation'.
    Bierze listę liczb i proponuje nowe stany, łącząc dwie liczby działaniem.
    """
    next_steps = []
    
    # Próbujemy połączyć każdą liczbę z każdą inną
    for i in range(len(current_numbers)):
        for j in range(len(current_numbers)):
            if i == j: continue # Nie łączymy liczby samej ze sobą
            
            a = current_numbers[i]
            b = current_numbers[j]
            
            # Reszta liczb (których nie ruszamy)
            remaining = [current_numbers[k] for k in range(len(current_numbers)) if k != i and k != j]
            
            # Generujemy możliwe operacje (Myśli)
            # 1. Dodawanie
            next_steps.append((remaining + [a + b], f"{a} + {b} = {a+b}"))
            # 2. Mnożenie
            next_steps.append((remaining + [a * b], f"{a} * {b} = {a*b}"))
            # 3. Odejmowanie (a-b)
            if a > b: # Uproszczenie: unikamy ujemnych dla czytelności
                next_steps.append((remaining + [a - b], f"{a} - {b} = {a-b}"))
            # 4. Dzielenie (tylko jeśli bez reszty)
            if b != 0 and a % b == 0:
                next_steps.append((remaining + [int(a / b)], f"{a} / {b} = {int(a/b)}"))
                
    return next_steps # Lista krotek: ([nowe_liczby], "opis kroku")

# Test generatora
kroki = get_possible_next_steps([4, 9, 10, 13])
print(f"Wymyślono {len(kroki)} możliwych pierwszych ruchów.")
print("Przykłady:")
for k in kroki[:3]:
    print(f" - {k[1]} -> Zostają: {k[0]}")

Wymyślono 30 możliwych pierwszych ruchów.
Przykłady:
 - 4 + 9 = 13 -> Zostają: [10, 13, 13]
 - 4 * 9 = 36 -> Zostają: [10, 13, 36]
 - 4 + 10 = 14 -> Zostają: [9, 13, 14]


## Algorytm Przeszukiwania (BFS - Breadth First Search)

To jest serce **Tree of Thoughts**.
Nie idziemy "na pałę" w głąb (jak zwykły Chain of Thought).
1.  Generujemy wszystkie możliwości dla kroku 1.
2.  (Opcjonalnie) Oceniamy je i odrzucamy słabe (Pruning).
3.  Dla tych co zostały, generujemy krok 2.

Stworzymy system, który trzyma **historię myśli** (Path).

In [3]:
def solve_tree_of_thoughts(numbers, target=24):
    # Kolejka stanów: (aktualne_liczby, historia_kroków)
    queue = [(numbers, [])]
    
    solutions = []
    
    # Pętla BFS (Szukamy rozwiązania poziom po poziomie)
    while queue:
        current_nums, history = queue.pop(0) # Pobieramy najstarszy element
        
        # SPRAWDZENIE SUKCESU
        # Jeśli została 1 liczba i jest to 24 -> Mamy to!
        if len(current_nums) == 1:
            if current_nums[0] == target:
                return history # Zwracamy zwycięską ścieżkę
            continue # Jak nie 24, to ślepa uliczka
            
        # GENEROWANIE NOWYCH MYŚLI
        possible_moves = get_possible_next_steps(current_nums)
        
        # (Tu normalnie byłby krok EWALUACJI przez LLM: "Czy ten ruch ma sens?")
        # My zrobimy prostą heurystykę: nie dodajemy do kolejki, jeśli nie ma sensu.
        
        for next_nums, step_desc in possible_moves:
            # Tworzymy nową historię
            new_history = history + [step_desc]
            # Dodajemy do kolejki do sprawdzenia w przyszłości
            queue.append((next_nums, new_history))
            
    return None # Nie znaleziono

print("Algorytm ToT gotowy.")

Algorytm ToT gotowy.


In [4]:
# URUCHOMIENIE
print(f"Rozwiązywanie dla: {input_numbers}...")

solution_path = solve_tree_of_thoughts(input_numbers)

if solution_path:
    print("\n✅ ZNALEZIONO ROZWIĄZANIE (Tree of Thoughts):")
    for i, step in enumerate(solution_path):
        print(f"Krok {i+1}: {step}")
else:
    print("❌ Nie udało się znaleźć rozwiązania.")

Rozwiązywanie dla: [4, 9, 10, 13]...

✅ ZNALEZIONO ROZWIĄZANIE (Tree of Thoughts):
Krok 1: 9 + 10 = 19
Krok 2: 19 - 13 = 6
Krok 3: 4 * 6 = 24


## Wersja LLM (Jak to wygląda w kodzie produkcyjnym?)

Gdybyśmy używali prawdziwego LLM (np. GPT-4), kod wyglądałby tak:

```python
# Pseudokod Tree of Thoughts z LLM

def generate_thoughts(state):
    prompt = f"Masz liczby {state}. Wypisz 3 możliwe kolejne kroki."
    return gpt4.complete(prompt) # Zwraca ["10-4=6", "13+9=22", ...]

def evaluate_thoughts(thoughts):
    prompt = f"Oceń, czy te kroki przybliżają nas do 24: {thoughts}. Daj punktację 0-1."
    return gpt4.complete(prompt) # Zwraca [0.9, 0.1, ...]

# Pętla sterująca
current_states = [start]
for step in range(3):
    new_states = []
    for s in current_states:
        thoughts = generate_thoughts(s)  # LLM wymyśla
        scores = evaluate_thoughts(thoughts) # LLM ocenia
        
        # Wybieramy tylko najlepsze myśli (Pruning)
        best_thoughts = [t for t, s in zip(thoughts, scores) if s > 0.7]
        new_states.extend(best_thoughts)
    
    current_states = new_states

## 🧠 Podsumowanie: Myślenie Systemu 2

Daniel Kahneman w książce "Pułapki myślenia" opisał dwa systemy:
1.  **System 1:** Szybki, instynktowny (To jest zwykły ChatGPT).
2.  **System 2:** Wolny, analityczny, planujący (To jest Tree of Thoughts).

**Inżynieria Promptów** w 2025 roku nie polega na szukaniu "magicznych słów".
Polega na budowaniu **architektury przepływu myśli** (Flow Engineering), gdzie wymuszamy na modelu System 2, każąc mu generować alternatywy i je oceniać przed podjęciem decyzji.